In [ ]:
# This file is used by get_new_raw_file.py to create a report after checking a new download.  To make this work nicely,
# you should use the 'hide-input-all' nbextension and before get_new_raw_file, enable hide all, reset and clear
# all cells, save the sheet, and CLose and Halt.  

In [ ]:
from math import log10, floor
def round_sig(x, sig=2):
    try:
        if abs(x)>=1:
            out =  int(round(x, sig-int(floor(log10(abs(x))))-1))
            return f"{out:,d}" # does the right thing with commas
        else: # fractional numbers
            return str(round(x, sig-int(floor(log10(abs(x))))-1))
    except:
        return x

In [ ]:
#preamble to analysis
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import seaborn as sns
import matplotlib.ticker
from IPython.display import Markdown as md
from time import sleep
from datetime import datetime, timedelta
now = datetime.now()
one_year_ago = now-timedelta(days=365)
today = str(datetime.today())
today = today.split()[0]

In [ ]:
md(f'# FracFocus bulk download summary report for {today}')

**These reports are now generated about once each week.** If you need the reports more frequently, let me know...

Note that if you have visited this page before, you may need to clear your browser's queue for the most recent results.  For many browsers, just type 'Ctrl-F5'.

The following data are from the most recently **published** fracking events. These are still raw names and numbers and have not yet been checked for validity. To see cleaned data set, [go here.](https://qbobioyuz1dh57rst8exeg-on.drv.tw/open_FF_catalog/)

In many cases below, individual fracking disclosures are identified by APINumber.  If you are interested in seeing the details of the raw data, use that APINumber at the FracFocus ["Find a Well" site](https://fracfocusdata.org/DisclosureSearch/Search.aspx).  This search site will deliver pdf files of individual fracking events to your computer with most of the same raw data available used here from the bulk download.

In [ ]:
lastrunfn = 'last_daily_run.txt'
try:
    with open(lastrunfn,'r') as f:
        lastrun = f.readline().strip()
except:
    lastrun = ' ----- '

In [ ]:
md('***')

In [ ]:
sources = './sources/'
datefn= './sources/upload_dates.csv'
currfn = 'testData'
outdir = './out/'
tempfolder = './tmp/'
webworkfolder = './website_gen/'
zfilename = 'testData'

updates = pd.read_csv(datefn)
updates['dt_added'] = pd.to_datetime(updates.date_added)
updates['days_old'] = (now - updates.dt_added).dt.days
new_upk = updates[updates.date_added==today].UploadKey.tolist()
month_upk = updates[updates.days_old<31].UploadKey.tolist()
#print(new_upk)
#md(f'### Number of new disclosures added yesterday: {len(new_upk)}')
md(f'### Number of new disclosures added since last download ({lastrun}): {len(new_upk)}')


In [ ]:
# First lets make today's list

import core.Construct_set as const_set
t = const_set.Construct_set(fromScratch=False,
                            zfilename=zfilename,
                            sources=sources,
                            outdir=outdir,
                            tempfolder=tempfolder).get_full_set();
with open(lastrunfn,'w') as f:
    f.write(f'{today}\n')
df = t.tables['allrec'].get_df()
# locat = t.get_df_location()[['UploadKey','StateName','CountyName',
#                              'iOperatorName','iUploadKey','TotalBaseWaterVolume',
#                              'FederalWell']]
locat = t.tables['event'].get_df()[['UploadKey','StateName','CountyName','APINumber',
                                    'iOperatorName','iUploadKey','TotalBaseWaterVolume',
                                    'FederalWell','IndianWell','JobEndDate']]
df = pd.merge(df,locat,on='iUploadKey',how='left')
casdf = t.tables['cas'].get_df()
df = pd.merge(df,casdf,on='iCASNumber',how='left')

df['end_date'] = df.JobEndDate.str.split().str[0]
df['date'] = pd.to_datetime(df.JobEndDate,errors='coerce')
todaydf = df[df.UploadKey.isin(new_upk)]
opdf = t.tables['operator'].get_df()
todaydf = pd.merge(todaydf,opdf,on='iOperatorName',how='left')
# df.columns

In [ ]:
# fetch proprietary labels
caslab = pd.read_csv('./sources/cas_labels.csv')
caslab.proprietary = np.where(caslab.proprietary==1,True,False)
prop_lab = list(caslab[caslab.proprietary].clean.unique())
df['proprietary'] = df.CASNumber.str.strip().str.lower().isin(prop_lab)

# fetch authenticated cas numbers
casref = pd.read_csv('./sources/CAS_ref_and_names.csv',quotechar='$')
cas_ok = list(casref.cas_number.unique())
df['clean_cas'] = df.CASNumber.str.strip().str.lower().isin(cas_ok)

In [ ]:
gb = todaydf.groupby(['UploadKey','StateName'],as_index=False)[['CountyName',
                                                           'OperatorName','APINumber',
                                                  'TotalBaseWaterVolume',
                                                  'FederalWell','IndianWell']].first()
tmp1 = gb.groupby(['StateName','CountyName','OperatorName'],as_index=False)['UploadKey'].count()
tmp1.rename({'UploadKey':'num_new_Disclosures'},inplace=True,axis=1)
tmp2 = gb.groupby(['StateName','CountyName','OperatorName'],
          as_index=False)['TotalBaseWaterVolume'].mean()
tmp2.rename({'TotalBaseWaterVolume':'mean_Water_Used_gal'},axis=1,inplace=True)
tmp2.mean_Water_Used_gal = tmp2.mean_Water_Used_gal.map(lambda x: round_sig(x,3))
out = pd.merge(tmp1,tmp2,on=['StateName','CountyName','OperatorName'],how='left')

if len(out)>0:
    display(md("""The following list is the most recently published fracking events. Note that these are still
raw names and numbers and have not yet been checked for validity."""))
    display(out)


In [ ]:
if len(gb)>0:
    gb['Water_vol_gallons'] = gb.TotalBaseWaterVolume.map(lambda x: round_sig(x,3))
    if gb.FederalWell.sum()>0:
#        display(md('# Disclosures published yesterday for fracking on US Federal lands'))
        display(md('# Disclosures published recently for fracking on US Federal lands'))
        display(gb[gb.FederalWell][['StateName','CountyName','OperatorName',
                                    'APINumber','Water_vol_gallons']])
    else:
        display(md('None of the disclosures above are on Federal Lands.'))


In [ ]:
if len(gb)>0:
    gb['Water_vol_gallons'] = gb.TotalBaseWaterVolume.map(lambda x: round_sig(x,3))
    if gb.IndianWell.sum()>0:
#        display(md('# Disclosures published yesterday for fracking on US Federal lands'))
        display(md('# Disclosures published recently labeled as "Indian Well"'))
        display(gb[gb.IndianWell][['StateName','CountyName','OperatorName',
                                   'APINumber','Water_vol_gallons']])
    else:
        display(md('None of the disclosures above are of "Indian Wells."'))


In [ ]:
monthdf = df[df.UploadKey.isin(month_upk)]
# opdf = t.tables['operator'].get_df()
monthdf = pd.merge(monthdf,opdf,on='iOperatorName',how='left')
monthdf = pd.merge(monthdf,updates,on='UploadKey',how='left')
# monthdf.columns

# End dates for disclosures published in the past 30 days

In [ ]:
gb = monthdf.groupby('UploadKey',as_index=False)[['date','APINumber','OperatorName']].first()
ax = gb.date.hist()
ax.set_title('Final dates for fracking jobs published in the last month')
ax.set_ylabel = ('Number of disclosures published in last 30 days')

## Jobs finished more than a year ago but just published

In [ ]:
old = gb[gb.date<one_year_ago]
display(old[['date','APINumber','OperatorName']].sort_values('date').head(30))

# Proprietary Claims

In [ ]:
# proprietary summaries
gb = monthdf.groupby('UploadKey',as_index=False)[['proprietary','clean_cas']].sum()
gb['fraction_prop'] = gb.proprietary/(gb.clean_cas+gb.proprietary)
ax = gb.fraction_prop.hist()
ax.set_title('Fraction of disclosure records that are claimed as "proprietary"');

## Recent disclosure with largest fractions of PROPRIETARY claims

In [ ]:
mg = pd.merge(gb,monthdf.groupby('UploadKey',as_index=False)[['APINumber','OperatorName','StateName']].first(),
              on='UploadKey',how='left')
display(mg[['fraction_prop','APINumber','OperatorName','StateName']].sort_values('fraction_prop',ascending=False).head(50))

# Water use reported in disclosures in the past 30 days

In [ ]:
import seaborn as sns
sns.set(style="whitegrid")
monthdf = monthdf.groupby('UploadKey',as_index=False)[['TotalBaseWaterVolume','StateName']].first()
sn = monthdf.groupby('StateName',as_index=False)['UploadKey'].count().astype('str')
sn.columns = ['StateName', 'cnt']
monthdf = pd.merge(monthdf,sn,on='StateName',how='left')
monthdf['State_Num'] = monthdf.StateName+'__'+monthdf.cnt
monthdf = monthdf.sort_values('cnt',ascending=False)
fig = plt.figure(figsize=(16,10))
ax = sns.stripplot(monthdf.TotalBaseWaterVolume,monthdf.State_Num,jitter=.2,alpha=.4,size=10)
plt.xlabel(f'water volume (gallons):',fontsize=14);
plt.title(f'Water Use for events published in the last 30 days',fontsize=16)
ax.set(xscale='log')
ax.set(xlim=(max(1000,monthdf.TotalBaseWaterVolume.min()),monthdf.TotalBaseWaterVolume.max()*1.1))
ax.grid(axis='y')
ax.tick_params(axis="x", labelsize=14)
ax.tick_params(axis="y", labelsize=14)
locmaj = matplotlib.ticker.LogLocator(base=10,subs='all') 
ax.xaxis.set_major_locator(locmaj)

# Disclosures published in the past 30 days with suspect values

## Zero or no reported Base Water Use:

In [ ]:
monthdf = df[df.UploadKey.isin(month_upk)]
monthdf = pd.merge(monthdf,opdf,on='iOperatorName',how='left')
monthdf = pd.merge(monthdf,updates,on='UploadKey',how='left')

In [ ]:
gb = monthdf.groupby('UploadKey',as_index=False)[['TotalBaseWaterVolume','APINumber',
                                                  'StateName','OperatorName','end_date']].first()
gb = gb.drop('UploadKey',axis=1)
display(gb[gb.TotalBaseWaterVolume==0].sort_values(['StateName','TotalBaseWaterVolume']))

## Base Water Use is greater than 30 million gallons:

In [ ]:
gb = monthdf.groupby('UploadKey',as_index=False)[['TotalBaseWaterVolume','APINumber',
                                                  'StateName','OperatorName','end_date']].first()
gb = gb.drop('UploadKey',axis=1)
gb['Probable Error'] = np.where(gb.TotalBaseWaterVolume>75000000,'<<<<<<<<<','')
display(gb[gb.TotalBaseWaterVolume>30000000].sort_values(['StateName','TotalBaseWaterVolume']))

## Sum of record percentages don't add to roughly 100%

In [ ]:
gb = monthdf.groupby('UploadKey',as_index=False)[['APINumber',
                                                  'StateName','OperatorName']].first()
gb1 = monthdf.groupby('UploadKey',as_index=False)[['PercentHFJob']].sum()
mg = pd.merge(gb1,gb,on='UploadKey')
mg = mg.drop('UploadKey',axis=1)

### Percent sum is less than 90 %

In [ ]:
display(mg[mg.PercentHFJob<90].sort_values('PercentHFJob'))

### Percent sum is greater than 110 %

In [ ]:
display(mg[mg.PercentHFJob>110].sort_values('PercentHFJob',ascending=False))

# New disclosures published per month since Jan. 2019

In [ ]:
updates['added'] = updates.dt_added.apply( lambda updates : datetime(year=updates.year, month=updates.month, day=updates.day))
updates.set_index(updates["added"],inplace=True)
updates.drop('added',axis=1,inplace=True)
counts = updates.resample('M').count()
counts.reset_index(inplace=True)
counts[counts.added.dt.year>2018].plot('added','UploadKey', title='Number of new disclosures');
#counts.head()
